In [42]:
import simpy
import random

In [48]:
# Create a Hospital class. It has limited beds ('NUM_BEDS') to treat patients in parallel.
# Patients wait for an available bed. When they get one they occupy it for a random time between 1 and 10.

class Hospital(object):
    def __init__(self, env, NUM_BEDS):
        self.env = env
        self.bed = simpy.Resource(env, NUM_BEDS)

    def treatment(self, patient):
        yield self.env.timeout(random.randint(1, 10))
        print("%d%% probability of %s returning to hospital" %
              (random.randint(1, 99), patient))

In [49]:
# Defines a patient which requires a bed upon arriving at hospital. 

def patient(env, name, cw):
    print('%s arrives at the hospital at %.2f.' % (name, env.now))
    with cw.bed.request() as request:
        yield request

        print('%s gets a bed at %.2f.' % (name, env.now))
        yield env.process(cw.treatment(name))

        print('%s leaves the hospital at %.2f.' % (name, env.now))

In [50]:
# create the enviroment with 4 initial patients and keep generating more patients every random
# interval between 1 and 5 while the simulation is running.

def setup(env, NUM_BEDS):
    hospital = Hospital(env, NUM_BEDS)

    for i in range(4):
        env.process(patient(env, 'Patient %d' % i, hospital))

    while True:
        yield env.timeout(random.randint(1, 5))
        i += 1
        env.process(patient(env, 'Patient %d' % i, hospital))

In [54]:
# input parameters

NUM_BEDS = 3  
SIM_TIME = 20 # days, for example


# initialise enviroment

env = simpy.Environment()
env.process(setup(env, NUM_BEDS))

<Process(setup) object at 0x10b90db38>

In [55]:
# execute simulation

env.run(until=SIM_TIME)

Patient 0 arrives at the hospital at 0.00.
Patient 1 arrives at the hospital at 0.00.
Patient 2 arrives at the hospital at 0.00.
Patient 3 arrives at the hospital at 0.00.
Patient 0 gets a bed at 0.00.
Patient 1 gets a bed at 0.00.
Patient 2 gets a bed at 0.00.
Patient 4 arrives at the hospital at 5.00.
52% probability of Patient 0 returning to hospital
Patient 0 leaves the hospital at 5.00.
Patient 3 gets a bed at 5.00.
29% probability of Patient 2 returning to hospital
Patient 2 leaves the hospital at 7.00.
Patient 4 gets a bed at 7.00.
66% probability of Patient 1 returning to hospital
Patient 5 arrives at the hospital at 10.00.
12% probability of Patient 4 returning to hospital
Patient 1 leaves the hospital at 10.00.
Patient 4 leaves the hospital at 10.00.
Patient 5 gets a bed at 10.00.
15% probability of Patient 3 returning to hospital
20% probability of Patient 5 returning to hospital
Patient 3 leaves the hospital at 11.00.
Patient 5 leaves the hospital at 11.00.
Patient 6 arrive